### Basic ChatCompletion API
The purpose of the completion API is to provide a response to complete a prompt. 

See docs: https://platform.openai.com/docs/api-reference/chat/create

Let's call the ChatCompletion API using gpt-3.5-turbo. 

In [1]:
pip install openai && pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
from dotenv import dotenv_values

In [3]:
# Set the Open API key from the .env file
config = dotenv_values(".env")
client = OpenAI(
  api_key=config["OPEN_API_KEY"]
)

In [35]:
# Create a completion model using gpt-3.5-turbo
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    { "role": "user", "content": "The dog says " } # role = "user" means you or who is chatting or who is asking the question
  ],
)

In [39]:
print(completion)

ChatCompletion(id='chatcmpl-9vBVSY2lkCyj5gxp7fyc3P71pmn4B', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"woof"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723415954, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=11, total_tokens=15))


### Analysis of ChatCompletion API
The result from the prompt "The dog says " was "woof" from the gpt-3.5-turbo (defaulted to gpt-3.5-turbo-0125) model. If you check your OpenAI API usage, you'll see the billing usage of the API.

As of today (08/11/2024), the pricing for gpt-3.5-turbo-0125 is `$0.50 / 1M tokens` for input and `$1.50 / 1M tokens` for output.

See billing usage in the website https://platform.openai.com/organization/usage.

### max_tokens parameter
The max_tokens parameter specifies the maximum number of tokens in the response. For more information about tokens, read the Tokens.md file.

See docs about max_tokens: https://platform.openai.com/docs/api-reference/completions/create#completions-create-max_tokens

In [7]:
# Text completion uses max_tokens of 16 in the response
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    { "role": "user", "content": "The top 10 places to visit are: " }
  ],
  max_tokens=16
)

In [8]:
print(completion)

ChatCompletion(id='chatcmpl-9vBxl7G2L4swedDyFOmMQcFa3soj5', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='1. The Grand Canyon, Arizona, USA\n2. Machu Picchu', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723417709, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=17, total_tokens=33))


### Analysis of max_tokens
We asked the model to predict the top 10 places to visit, but it only gave us 2 places. This is because we set the `max_tokens=16` in the completions API request, so the model stopped prematurely with a `finish_reason` of 'length'.

### Stop sequence
Stop sequences are useful to stop the model from generating text after a specific word or phrase.

See docs about stop sequences: https://platform.openai.com/docs/api-reference/completions/create#completions-create-stop

In [4]:
# Text completion with stop sequencing of "5."
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    { "role": "user", "content": "The top 10 places to visit are: " }
  ],
  stop="5.",
)

In [6]:
print(completion)

ChatCompletion(id='chatcmpl-9xdYzThYuxFgNYpreL2Mtn59DjHwS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Paris, France\n2. New York City, USA\n3. Rome, Italy\n4. Tokyo, Japan\n', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724000461, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=17, total_tokens=43))


In [19]:
# Text completion with a chat between the user and assistant and stop before saying another sentence.
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    { "role": "user", "content": "User: Hi, how are you?" },
    { "role": "assistant", "content": "Chatbot: I'm good!" },
    { "role": "user", "content": "User: Tell me about a time you visited Nepal." },
    { "role": "assistant", "content": "Chatbot: When I was 5 years old, I visited Nepal and was in Pokhara." },
    { "role": "user", "content": "User: What did you do there?" },
    { "role": "assistant", "content": "Chatbot: I went to the lake and went trekking with my friends." },
    { "role": "user", "content": "User: That sounds fun, how long did it take?" },
  ],
  stop=[".", "!", "?"]
)

In [20]:
print(completion)

ChatCompletion(id='chatcmpl-9xdqM4xNIZzsZowSCPZy5onYOnhvr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Chatbot: We trekked for about 2 hours before heading back to our hotel for the evening', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724001538, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=114, total_tokens=135))


### Analysis of stop sequencing
In the first example, we asked the model to predict the top 10 places to visit, but it stopped right before generating the 5th item because we set the `stop="5."` in the completions API request.

In the second example, we asked the model to explain his trip to Pokhara and stopped it before saying another sentence because we set the `stop=[".", "!", "?"]` in the completions API request.